# This Notebook explores the SCAR GeoMAP dataset released in 2019
## Cox S.C., Smith Lyttle B. and the GeoMAP team (2019). Lower Hutt, New Zealand. GNS Science. Release v.201907.
### [Data Available Here](https://data.gns.cri.nz/ata_geomap/index.html?content=/mapservice/Content/antarctica/www/index.html)
### Notebook by Sam Elkind

Initially, I'll look at the data in terms of polygon counts. This section will be focused on examining the data schema and frequency of values occurring within specific fields. This investigation will focus on finding inconsistencies in the data attribution, but also could stimulate some discussion regarding relationships between columns.

Next, I'll look at the data in terms of polygon area and data attribution. How much surface water has been mapped? How much till has been mapped? How much outcropping rock is of Jurassic age?

### Configure packages, paths, and load data

In [1]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import pprint as pp
from tabulate import tabulate

In [2]:
def plot_value_counts(field_name, values_to_plot, counts, counts_norm):
    fig, ax = plt.subplots(2, 1, figsize=(30,15))
    fig.tight_layout(pad=2.0)
    fig.subplots_adjust(top=.94)
    fig.suptitle(f"Frequency of {field_name} values", size=18)

    ax[0].set_title(field_name)
    ax[1].set_title(f"{field_name} normalized")
    for i, v in enumerate(counts[:values_to_plot]):
        ax[0].text(i - .5, v, str(v), color='black', fontweight='bold')
    for i, v in enumerate(counts_norm[:values_to_plot]):
        ax[1].text(i - .5, v, f"{str(v * 100)[:3]}%", color='black', fontweight='bold')
    ax[0].bar(counts.index[:values_to_plot], counts[:values_to_plot])
    ax[1].bar(counts_norm.index[:values_to_plot], counts_norm[:values_to_plot])

In [3]:
geol_path = f"{os.getcwd()}/data/ATA_SCAR_GeoMAP_geology.gdb"
print(geol_path)

/home/gt/geomap/data/ATA_SCAR_GeoMAP_geology.gdb


In [4]:
data = gpd.read_file(geol_path)

# This notebook is to investigate the relationship between sourcecode and description

In [5]:
# helpers
display_cols = ["SOURCECODE", "DESCR", "MAPSYMBOL", "NAME", "SOURCE"]

In [6]:
src_codes = data["SOURCECODE"].unique()
descriptions = data["DESCR"].unique()

In [7]:
print(f"There are {len(src_codes)} unique source codes and {len(descriptions)} unique descriptions")

There are 801 unique source codes and 758 unique descriptions


In [8]:
src_code_description_pairs = data[display_cols].drop_duplicates(["SOURCECODE", "DESCR"])

In [9]:
print(f"There are {len(src_code_description_pairs)} sourcecode description pairs")

There are 852 sourcecode description pairs


### Let's look at source codes that have multiple descriptions and vice versa

In [12]:
src_code_descr_sets = sorted([data[data["SOURCECODE"] == i][display_cols].drop_duplicates("DESCR").sort_values(["DESCR", "SOURCE"]) for i in src_codes if len(data[data["SOURCECODE"] == i][display_cols].drop_duplicates("DESCR")) > 1], key=lambda x: len(x), reverse=True)
descr_src_code_sets = sorted([data[data["DESCR"] == i][display_cols].drop_duplicates("SOURCECODE").sort_values(["SOURCECODE", "SOURCE"]) for i in descriptions if len(data[data["DESCR"] == i][display_cols].drop_duplicates("SOURCECODE")) > 1], key=lambda x: len(x), reverse=True)

In [13]:
print(len(src_code_descr_sets))
print(len(descr_src_code_sets))

42
60


In [14]:
for i in src_code_descr_sets:
    print(i["DESCR"].values)
    display(i)

['Locally derived glacial till in moraines and rock glaciers, variably weathered, inferredto be derived from existing alpine glaciers'
 'Locally derived glacial till in moraines and rock glaciers, variably weathered; inferred to be derived from existing alpine glaciers'
 'Locally derived glacial till in moraines and rock glaciers: variably weathered: inferred to be derived from existing alpine glaciers'
 'Undifferentiated local glacier till']


,SOURCECODE,DESCR,MAPSYMBOL,NAME,SOURCE
39753,Qtl,Locally derived glacial till in moraines and r...,Qs,older local glacial till,GeoMAP
40877,Qtl,Locally derived glacial till in moraines and r...,Qs,local glacial till,GeoMAP
43670,Qtl,Locally derived glacial till in moraines and r...,Qs,older local glacial till,Grindley & Laird 1969
75549,Qtl,Undifferentiated local glacier till,Qs,Quaternary till local glacier,GeoMAP


['Holocene local glacier till'
 'Locally derived variably weathered till in moraines and rock glaciers, usually associated with alpine glaciers'
 'Locally-derived variably weathered till in moraines and rock glaciers, usually associated with alpine glaciers'
 'Young locally derived glacial till in moraines and rock glaciers: associated with existing alpine glaciers']


,SOURCECODE,DESCR,MAPSYMBOL,NAME,SOURCE
75261,Htl,Holocene local glacier till,Hs,Holocene till local glacier,GeoMAP
43662,Htl,Locally derived variably weathered till in mor...,Hs,Holocene local glacier till,Grindley & Laird 1969
40124,Htl,Locally-derived variably weathered till in mor...,Hs,Holocene local glacier till,GeoMAP
40788,Htl,Young locally derived glacial till in moraines...,Hs,Holocene local glacier till,GeoMAP


['Supra-glacial till'
 'Surficial till on top of glacial ice and snow, may include active or mobile ice-cored till'
 'Surficial till on top of glacial ice and snow: may include some young active or mobile ice-cored till']


...

,SOURCECODE,DESCR,MAPSYMBOL,NAME,SOURCE
75777,Qk,"Loose lacustrine sand, silt, mud.",Qs,lake deposits,GeoMAP
73018,Qk,"Loose lacustrine sand, silt, mud; locally mixe...",Qk,lake deposits,Mayewski et al. 1979a


['Beds moderately to strongly folded, secondary foliation common. Low grade metamorphism except near intrusive bodies.'
 'Dark-gray cordierite and hypersthene-bearing porphyry; probably a large, thick sill; principal rock of the Theil Mountains; age relationship to the Patuxent Formation unknown.']


,SOURCECODE,DESCR,MAPSYMBOL,NAME,SOURCE
76871,pCm,"Beds moderately to strongly folded, secondary ...",Rzp,Precambrian metasedimentary rocks,Craddock 1972
76874,pCm,Dark-gray cordierite and hypersthene-bearing p...,Rzg,Quartz Monzonite Porphyry,Schmidt and Ford 1969


['Orthopyroxene-biotite-quartz-plagioclase gneiss (tonalitic orthogneiss); minor granodioritic to granitic orthogneiss, garnet-biotite-quartz-feldspar gneiss, metasediments, and mafic granulite'
 'Orthopyroxene-quartz-plagioclase gneiss (tonalitic orthogneiss); minor granitic orthogneiss, biotite-garnet-quartz-feldspar- gneiss, metasediments, and mafic granulite.']


,SOURCECODE,DESCR,MAPSYMBOL,NAME,SOURCE
95112,Ppp,Orthopyroxene-biotite-quartz-plagioclase gneis...,Rzn,None,Sheraton 1985. Geology of Enderby Land and Wes...
95135,Ppp,Orthopyroxene-quartz-plagioclase gneiss (tonal...,Rzn,None,"AGSO, Bedrock Geology of the Bunger Hills-Denm..."


In [15]:
for i in descr_src_code_sets:
    print(i["SOURCECODE"].values)
    display(i)

['Hv_Cl002_3' 'Hv_HaC14a' 'Hv_LMA11b' 'Hv_Nar1a' 'Mev_WVF14b' 'Mev_WoF1b'
 'Pb_Hart97b' 'Pb_LMB12.3b' 'Pb_LMB16D.1c' 'Pb_LMB16b2' 'Pb_LeM033c'
 'Pb_LeM6b2' 'Pb_LeM7b3' 'Pb_Pan4b']


,SOURCECODE,DESCR,MAPSYMBOL,NAME,SOURCE
63456,Hv_Cl002_3,Trachyte,Czf,Hallett volcanic province,"Hornig & Wörner, 2003"
61040,Hv_HaC14a,Trachyte,Czf,Hallett volcanic province,"Hamilton, 1972 (C-14)"
73792,Hv_LMA11b,Trachyte,Czf,Hallett volcanic province,LeMasurier & Thomson 1990 (Fig A.1.1)
63800,Hv_Nar1a,Trachyte,Czf,Hallett volcanic province,Nardini et al. 2003
73798,Mev_WVF14b,Trachyte,Czf,Melbourne volcanic province,Wörner & Viereck 1989 Fig14
62049,Mev_WoF1b,Trachyte,Czf,Melbourne volcanic province,Wörner et al. 1989 Fig1
41844,Pb_Hart97b,Trachyte,Czf,Marie Byrd Land Volcanics: trachyte,Hart et al. 1997
42489,Pb_LMB12.3b,Trachyte,Czf,Marie Byrd Land Volcanics: trachyte,LeMasurier & Thomson 1990 (Fig B.12.3)
41955,Pb_LMB16D.1c,Trachyte,Czf,Marie Byrd Land Volcanics: trachyte,LeMasurier & Thomson 1990 (Fig B.16D.1)
42495,Pb_LMB16b2,Trachyte,Czf,Marie Byrd Land Volcanics: trachyte,LeMasurier & Thomson 1990 (Fig B.16B.1)


['Pb_LM84b' 'Pb_LMB14.2d' 'Pb_LMB16D.1d' 'Pb_LeM3b3' 'Pb_LeM4b3'
 'Pb_LeM6b3']


,SOURCECODE,DESCR,MAPSYMBOL,NAME,SOURCE
42095,Pb_LM84b,Marie Byrd Land Volcanics: undifferentiated,Czv,Marie Byrd Land Volcanics: undifferentiated,LeMasurier 1984
41970,Pb_LMB14.2d,Marie Byrd Land Volcanics: undifferentiated,Czv,Marie Byrd Land Volcanics: undifferentiated,LeMasurier & Thomson 1990 (Fig B.14.2)
41853,Pb_LMB16D.1d,Marie Byrd Land Volcanics: undifferentiated,Czv,Marie Byrd Land Volcanics: undifferentiated,LeMasurier & Thomson 1990 (Fig B.16D.1)
41801,Pb_LeM3b3,Marie Byrd Land Volcanics: undifferentiated,Czv,Marie Byrd Land Volcanics: undifferentiated,LeMasurier 2013 (Fig.3B)
41775,Pb_LeM4b3,Marie Byrd Land Volcanics: undifferentiated,Czv,Marie Byrd Land Volcanics: undifferentiated,LeMasurier 2013 (Fig.4B)
42938,Pb_LeM6b3,Marie Byrd Land Volcanics: undifferentiated,Czv,Marie Byrd Land Volcanics: undifferentiated,LeMasurier 2013 (Fig.6B)


['Mev_LMA61c' 'Pb_Kip14a' 'Pb_LMB14.2a' 'Pb_LeM4b1' 'Pb_LeM6b1' 'Pb_Pan4a']


...

,SOURCECODE,DESCR,MAPSYMBOL,NAME,SOURCE
84907,1040,"Dark-coloured, mafic and ultramafic rocks meta...",NEe,Banded gneisses (metavolcanics),Norwegian Polar Institute
84797,1045,"Dark-coloured, mafic and ultramafic rocks meta...",NEe,Mafic gneiss and schist/mafic lenses and dikes,Norwegian Polar Institute


['CGD' 'Qm']


,SOURCECODE,DESCR,MAPSYMBOL,NAME,SOURCE
92081,CGD,"Moraines, mostly ice-cored on tops or flanks o...",Qs,None,AGSO. Geology of the Northern Vestfold Hills (...
85580,Qm,"Moraines, mostly ice-cored on tops or flanks o...",Qs,,GeoMAP


['Ago' 'Atg']


,SOURCECODE,DESCR,MAPSYMBOL,NAME,SOURCE
85708,Ago,Medium grained orthopyroxene-clinopyroxene-pla...,An,Glider Lake sub-Suite of Tryne Mafic gneiss.,AGSO. Geology of the Northern Vestfold Hills (...
85865,Atg,Medium grained orthopyroxene-clinopyroxene-pla...,An,Tryne Mafic Gneiss,AGSO. Geology of the Northern Vestfold Hills (...
